### process era5 montly mean gpi

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
from datetime import datetime
from datetime import date
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm
import scipy.stats
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def plot_polygon(ax, sub_region):
    """
    https://groups.google.com/forum/#!topic/scitools-iris/LxR0EbQolyE
    Note:
    ----
    order is important:
        lower-left, upper-left, upper-right, lower-right
        2 -- 3
        |    |
        1 -- 4
    """
    # ax = fig.axes[0]
    lons = [sub_region.latmin, sub_region.latmin, sub_region.latmax, sub_region.latmax]
    lats = [sub_region.lonmin, sub_region.lonmax, sub_region.lonmax, sub_region.lonmin]
    ring = LinearRing(list(zip(lons, lats)))
    ax.add_geometries([ring], cartopy.crs.PlateCarree(), facecolor='b', edgecolor='black', alpha=0.5)
    return ax

def plot_GPI(GPI,latS,latN,lonW,lonE):
 
    import cartopy.crs as ccrs
    import matplotlib.pyplot as plt
    import matplotlib.ticker as mticker
    import cartopy.feature as cfeature
    from cartopy.mpl.gridliner import LongitudeFormatter, LatitudeFormatter

    from geocat.viz import util as gvutil
    from shapely.geometry.polygon import LinearRing

   
    minC = 0.
    maxC = 1.0 #100
    intC = .1  #10
   

 
    minC = 0.
    maxC = 20
    intC = 1
   

    levC = int((maxC-minC)/intC) + 1

# Generate figure (set its size (width, height) in inches)
    fig = plt.figure(figsize=(24, 6))

# Generate axes, using Cartopy, drawing coastlines, and adding features
    projection = ccrs.PlateCarree()

    ax = plt.axes(projection=projection)
    ax.coastlines(linewidths=0.5)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')

    # Import an NCL colormap
    #newcmp = gvcmaps.BlAqGrYeOrRe
    newcmp = 'YlOrBr'  #'YlGn' #'coolwarm'  #gvcmaps.ncl_default
    #newcmp = gvcmaps.BlAqGrYeOrRe
# Contourf-plot data

    contourLevels = np.arange(minC, maxC, intC)
    vmaxMap = GPI.plot.contourf(ax=ax,
                          transform=projection,
                          vmin=minC,
                          vmax=maxC,
                          levels=contourLevels,
                          cmap=newcmp,
                          add_colorbar=False)

    gvutil.set_axes_limits_and_ticks(ax,
                                     xlim=(lonW, lonE),
                                     ylim=(latS, latN),
                                     xticks=np.linspace(lonW,lonE , 6),
                                     yticks=np.linspace(latS, latN,3))

     # Use geocat.viz.util convenience function to add minor and major tick lines
    gvutil.add_major_minor_ticks(ax)

    # Use geocat.viz.util convenience function to make plots look like NCL plots by
    # using latitude, longitude tick labels
    gvutil.add_lat_lon_ticklabels(ax)
    # Remove the degree symbol from tick labels
    ax.yaxis.set_major_formatter(LatitudeFormatter(degree_symbol=''))
    ax.xaxis.set_major_formatter(LongitudeFormatter(degree_symbol=''))
   
    
    lons = [lonW, lonW, lonE, lonE]
    lats = [latS, latN, latN, latS]
    
    ring = LinearRing(list(zip(lons, lats)))
    ax.add_geometries([ring], ccrs.PlateCarree(), facecolor='none', edgecolor='black')

    
    # Add colorbar
    cbar = plt.colorbar(vmaxMap, orientation='horizontal',ticks=np.arange(minC, maxC, intC))
#cbar.ax.set_yticklabels([str(i) for i in np.arange(minC, maxC, intC)])
#cbar.ax.set_yticklabels(str(conto


In [ ]:
# for monthly mean data
# first set the years and months for the processing
yearStart = 1995
yearEnd   = 2021
dir = '/home/sagan/work/data/data100/data/era5/mon_gpi/'
filenames = [dir + str(year)  + '.nc' for year in range(yearStart,yearEnd+1)]
data = xr.open_mfdataset(filenames, parallel=True)
gpi = data.GPI
monthStart = 6
monthEnd   = 10
#--------------------------------------------------

# atlantic MDR
latS = 10.
latN = 22.
lonW = 275.
lonE = 345.

# epac 
latS = 5.
latN = 25.
lonW = 240.
lonE = 275.

gpiMean = gpi.where( (gpi['time'].dt.month>5) & (gpi['time'].dt.month<11) ).mean('time')
print("GPI = " , gpiMean.min().values, gpiMean.max().values)

plot_GPI(gpiMean,latS,latN,lonW-360.,lonE-360.)

In [ ]:

filenames='EN.nc'

data = xr.open_mfdataset(filenames, parallel=True)
gpiMean = data.GPI

# atlantic MDR
latS = -10.
latN =  30.
lonW =  40.
lonE =  85.

print("GPI = " , gpiMean.min().values, gpiMean.max().values)

plot_GPI(gpiMean,latS,latN,lonW,lonE)

In [ ]:

filenames='E11.nc'

data = xr.open_mfdataset(filenames, parallel=True)
gpiMean = data.GPI

# atlantic MDR
latS = -10.
latN =  30.
lonW =  40.
lonE =  85.

print("GPI = " , gpiMean.min().values, gpiMean.max().values)

plot_GPI(gpiMean,latS,latN,lonW,lonE)

In [ ]:


filenames='20190610.nc'

data = xr.open_mfdataset(filenames, parallel=True)
gpiMean = data.GPI

print("GPI = " , gpiMean.min().values, gpiMean.max().values)

plot_GPI(gpiMean,latS,latN,lonW,lonE)

In [ ]:
gpiTS = gpi.where( (gpi['time'].dt.month>5) & (gpi['time'].dt.month<11) ) \
.sel(latitude=slice(latN,latS), longitude=slice(lonW,lonE))\
.mean(dim=['latitude','longitude'])

gpiTS = gpiTS.to_dataframe() #.drop(['number', 'step', 'valid_time','surface'], axis=1)  
gpiTS['Year'] = gpiTS.index.year
gpiTS.index = gpiTS['Year']
gpiTS = gpiTS.groupby(gpiTS.index).mean()


In [ ]:
dataDir  = "~/work/data/data100/data/ibtracs/"
filename = "IBTrACS.since1980.v04r00.nc"

file = dataDir+filename

try:
    ds = xr.open_dataset(file)
except:
    print ("file not found. quitting code")
    quit()
    
print ("Ibtracs file found and opened")


# NOW just extract the first track point info for each TC track

numobs   = ds.numobs.astype(int) - 1
lifeTime = (ds.time[:,numobs] - ds.time[:,0])/ np.timedelta64(1, 'D')
print(lifeTime)


time   = ds.time[:,0].where(lifeTime>3.0, drop=True)


#ibtracs lat-lon are negative for the western hemisphere. so add 360 to make them
#positive
time['lon'] = time['lon']+ 360.



In [ ]:
tsub = time.where((time["time.year"] >= yearStart)&(time["time.year"] <= yearEnd) , drop=True)
# further subset for the month range
tsub = tsub.where((tsub["time.month"] >= monthStart)&(tsub["time.month"] <= monthEnd) , drop=True)

# subset tc genesis times for the lat-lob box defined
tsub   = tsub.where((tsub.lon>=lonW) & (tsub.lon<=lonE) & (tsub.lat>=latS) & (tsub.lat<=latN), drop=True)
#nobs   = numobs.where((time.lon>=lonW) & (time.lon<=lonE) & (time.lat>=latS) & (time.lat<=latN), drop=True)


In [ ]:

yearly = tsub.groupby("time.year").count()
yearly = yearly.to_dataframe() #.drop(['number', 'step', 'valid_time','surface'], axis=1)  


#print(yearly.index)
#yearly = yearly.drop(yearly.index[yearly.index>2018])
#print(yearly.index)
#====================================================================================


yearly['GPI'] = gpiTS['GPI']
yearly = yearly.rename({'time': 'count'}, axis=1)
print(yearly)


In [ ]:

yearly[['count', 'GPI']] = StandardScaler().fit_transform(yearly[['count', 'GPI']])

fig, ax =  plt.subplots(nrows=1, ncols=1)
ax=yearly.plot(y=["count", "GPI"], style=['o:','o--'], figsize=(15,8), ax=ax)
#ax.set(xlabel='Year', ylabel='Genesis Fraction')
ax.set_title("(a)", loc="left")
ax.set_xlabel('Year', fontsize=18)
ax.set_ylabel('Storm Number', fontsize=18) 
ax.legend(["count", "GPI"]);


In [ ]:
Y = yearly['count']
X = yearly['GPI']
X = np.array(X).reshape(-1,1)
degreePoly = 1
polynomial_features= PolynomialFeatures(degree=degreePoly)
print(polynomial_features)
xp = polynomial_features.fit_transform(X)

# Fit regression model
results = sm.OLS(Y, xp).fit()
# Inspect the results

print( results.params)
print(results.summary())
regEq = 'R$^2$ = {:.2f}'.format(results.rsquared)

#sns.set_theme(style="white")
sns.set_style('darkgrid')
fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(6.0,6.0)
ax=sns.regplot(y=yearly["count"], x=yearly["GPI"], lowess=True, scatter_kws={'facecolors':'none', 'alpha':0.4, 'edgecolor':"magenta"}, line_kws={'color': "black", 'alpha':.7}, ci=None )
# choose color for the lines which will also be used for the ci (translucent version of the color)
ax=sns.regplot(y=yearly["count"], x=yearly["GPI"], order=degreePoly, scatter_kws={'facecolors':'none', 'alpha':0.4, 'edgecolor':"magenta"}, line_kws={'color': "orange"}, ci=95) #, ci=None)
ax.set(ylim=(-2, 2), xlim=(-2,2))
ax.set_xlabel('GPI', fontsize=14)
ax.set_ylabel('storm count', fontsize=14) 

ax.text(0.95, 0.91, regEq,
        verticalalignment='top', horizontalalignment='right',
        transform=ax.transAxes,
        color='orange', fontsize=15)

ax.set_title("(a)", loc="left")

fig=ax.get_figure()
fig.savefig('epac_atl_N_scatter.png')



fig, ax = plt.subplots()
fig.set_size_inches(6.0,6.0)
ax = sns.residplot(y=yearly["count"], x=yearly["GPI"], order=degreePoly, color="magenta", scatter_kws={'alpha':0.1}, line_kws={'color': "black"},lowess=True)